In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split


In [2]:
data = torch.load('/Users/pinchichen/2025S lab/AI drug project/code/TrainingDataset/Kcat_Enzymatic_reaction.pt',weights_only=False)
data.head()

,ECNumber,Organism,Smiles,Substrate,Sequence,Raw Kd,Unit,Log Kd
0,4.2.3.4,Actinidia chinensis,C(C(C(C(COP(=O)(O)O)O)O)O)C(=O)C(=O)O,3-deoxy-D-arabino-heptulosonic acid 7-phosphate,MAAFSLSAKQILSPSTHRPSLSKTTTADSSLRFRNPHSLSLRCSSL...,20.1000,s^(-1),1.303196
1,4.2.3.4,Aspergillus nidulans,C(C(C(C(COP(=O)([O-])[O-])O)O)O)C(=O)C(=O)[O-],3-deoxy-D-arabino-heptulosonate 7-phosphate,MSNPTKISILGRESIIADFGLWRNYVAKDLISDCSSTTYVLVTDTN...,6.8000,s^(-1),0.832509
2,4.2.3.4,Neurospora crassa,C(C(C(C(COP(=O)(O)O)O)O)O)C(=O)C(=O)O,3-deoxy-D-arabino-heptulosonic acid 7-phosphate,MAEPISNPTRINILGKDNIIIDHGIWLNFVAQDLLQNIKSSTYILI...,19.0000,s^(-1),1.278754
3,2.1.1.255,Streptomyces coelicolor,C[S+](CCC(C(=O)[O-])N)CC1C(C(C(O1)N2C=NC3=C(N=...,S-Adenosyl-L-methionine,MTTETTTATATAKIPAPATPYQEDIARYWNNEARPVNLRLGDVDGL...,0.0075,s^(-1),-2.124939
4,2.1.1.255,Streptomyces coelicolor,CC(=CCCC(=C(C)COP(=O)(O)OP(=O)(O)O)C)C,(E)-2-Methylgeranyl diphosphate,MTTETTTATATAKIPAPATPYQEDIARYWNNEARPVNLRLGDVDGL...,0.0390,s^(-1),-1.408935


In [3]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load ChemRoBERTa tokenizer and model
chem_tokenizer = AutoTokenizer.from_pretrained("seyonec/ChemBERTa-zinc-base-v1")
chem_model = AutoModel.from_pretrained("seyonec/ChemBERTa-zinc-base-v1")

# Load ESM2 tokenizer and model
esm_tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t6_8M_UR50D")
esm_model = AutoModel.from_pretrained("facebook/esm2_t6_8M_UR50D")

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
def extract_chem_features(smiles):
    """Extract ChemRoBERTa features from SMILES."""
    try:
        tokens = chem_tokenizer(smiles, return_tensors="pt", padding=True, truncation=True)
        with torch.no_grad():
            embeddings = chem_model(**tokens).last_hidden_state.mean(dim=1).squeeze().numpy()
        return embeddings
    except:
        return np.zeros(768)  # Return zero vector if extraction fails

def extract_esm_features(sequence):
    """Extract ESM2 features from protein sequence."""
    try:
        tokens = esm_tokenizer(sequence, return_tensors="pt", padding=True, truncation=True)
        with torch.no_grad():
            embeddings = esm_model(**tokens).last_hidden_state.mean(dim=1).squeeze().numpy()
        return embeddings
    except:
        return np.zeros(768)  # Return zero vector if extraction fails

In [5]:
# Extract unique mols and proteins
unique_mols = data[['Smiles']].drop_duplicates()
unique_proteins = data[['Sequence']].drop_duplicates()

In [6]:
len(unique_mols), len(unique_proteins)

(2706, 7857)

In [7]:
# Extract features for unique mols
tqdm.pandas()  # Enable progress bar for pandas
unique_mols['metabolite_features'] = unique_mols['Smiles'].progress_apply(extract_chem_features)

100%|██████████| 2706/2706 [00:51<00:00, 52.76it/s]


In [8]:
# Extract features for unique proteins
unique_proteins['protein_features'] = unique_proteins['Sequence'].progress_apply(extract_esm_features)

100%|██████████| 7857/7857 [08:47<00:00, 14.90it/s]  


In [9]:
# Merge features back into the combined dataframe
data = data.merge(unique_mols, on='Smiles', how='left')
data = data.merge(unique_proteins, on='Sequence', how='left')

In [10]:
# adding 1e-9 to prevent error when doing logarithm
data['label'] = -np.log10(data['Raw Kd'] + 1e-9)

In [11]:
data.columns

Index(['ECNumber', 'Organism', 'Smiles', 'Substrate', 'Sequence', 'Raw Kd',
       'Unit', 'Log Kd', 'metabolite_features', 'protein_features', 'label'],
      dtype='object')

In [12]:
# Separate the dataset by unique proteins and drugs
unique_proteins = data['Sequence'].unique()
unique_mols = data['Smiles'].unique()
# Set the seed for reproducibility
torch.manual_seed(42)
# Function to perform a cold split
def cold_split(unique_items, test_size=0.2, val_size=0.1):
    train_items, test_items = train_test_split(unique_items, test_size=test_size, random_state=42)
    train_items, val_items = train_test_split(train_items, test_size=val_size / (1 - test_size), random_state=42)
    return train_items, val_items, test_items
# Cold split by proteins
train_proteins, val_proteins, test_proteins = cold_split(unique_proteins)
train_cold_protein = data[data['Sequence'].isin(train_proteins)]
val_cold_protein = data[data['Sequence'].isin(val_proteins)]
test_cold_protein = data[data['Sequence'].isin(test_proteins)]
# Cold split by molecules
train_mols, val_mols, test_mols = cold_split(unique_mols)
train_cold_mols = data[data['Smiles'].isin(train_mols)]
val_cold_mols = data[data['Smiles'].isin(val_mols)]
test_cold_mols = data[data['Smiles'].isin(test_mols)]

In [13]:
def df2array(df):
    X = np.array([
    np.concatenate([m, p])
    for m, p in zip(df['metabolite_features'], df['protein_features'])])
    y = df['label']
    return X, y

train_X, train_y = df2array(train_cold_protein)
val_X, val_y = df2array(val_cold_protein)
test_X, test_y = df2array(test_cold_protein)

train_X_mols, train_y_mols = df2array(train_cold_mols)
val_X_mols, val_y_mols = df2array(val_cold_mols)
test_X_mols, test_y_mols = df2array(test_cold_mols)

In [14]:
#model initialization
import xgboost as xg
xgb = xg.XGBRegressor()
xgb_2 = xg.XGBRegressor()

In [15]:

'''
#Grid search
#ref: https://xgboost.readthedocs.io/en/stable/parameter.html
from sklearn.model_selection import GridSearchCV
param_grid = dict(
    max_depth = [5,10,15,20,None],
    n_estimators = [50,100,150,200]
)
regressor = GridSearchCV(estimator = xgb, param_grid = param_grid, cv = pds, verbose = 3, n_jobs = -1)
'''

#Train the model
xgb.fit(train_X,train_y)
xgb_2.fit(train_X_mols, train_y_mols)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [17]:
# Train
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, median_absolute_error, explained_variance_score
from scipy.stats import pearsonr

def metrics_cal(model,X,y):
    y_pred = model.predict(X)
    mse = mean_squared_error(y, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y, y_pred)
    r2 = r2_score(y, y_pred)
    pearson_corr, _ = pearsonr(y, y_pred)
    median_ae = median_absolute_error(y, y_pred)
    explained_var = explained_variance_score(y, y_pred)
    return mse, rmse, mae, r2, pearson_corr, median_ae, explained_var


train_mse, train_rmse, train_mae, train_r2, train_pearson_corr, train_median_ae, train_explained_var = metrics_cal(xgb, train_X, train_y)
val_mse, val_rmse, val_mae, val_r2, val_pearson_corr, val_median_ae, val_explained_var = metrics_cal(xgb, val_X, val_y)
test_mse, test_rmse, test_mae, test_r2, test_pearson_corr, test_median_ae, test_explained_var = metrics_cal(xgb, test_X, test_y)

In [18]:
train_mse_mols, train_rmse_mols, train_mae_mols, train_r2_mols, train_pearson_corr_mols, train_median_ae_mols, train_explained_var_mols = metrics_cal(xgb_2, train_X_mols, train_y_mols)
val_mse_mols, val_rmse_mols, val_mae_mols, val_r2_mols, val_pearson_corr_mols, val_median_ae_mols, val_explained_var_mols = metrics_cal(xgb_2, val_X_mols, val_y_mols)
test_mse_mols, test_rmse_mols, test_mae_mols, test_r2_mols, test_pearson_corr_mols, test_median_ae_mols, test_explained_var_mols = metrics_cal(xgb_2, test_X_mols, test_y_mols)

In [19]:
# append the performance to the csv file
df = {
    'Model':['XGB','XGB','XGB'],
    'Dataset':['Train','Validation','Test'],
    'MSE':[train_mse,val_mse,test_mse],
    'RMSE':[train_rmse,val_rmse,test_rmse],
    'MAE':[train_mae,val_mae,test_mae],
    'R2':[train_r2,val_r2,test_r2],
    'Pearson':[train_pearson_corr,val_pearson_corr,test_pearson_corr],
    'Median_AE':[train_median_ae,val_median_ae,test_median_ae],
    'Explained_VAR':[train_explained_var,val_explained_var,test_explained_var],
    'Dataspliting Mode':['cold protein','cold protein','cold protein']
}
df = pd.DataFrame(df)

df.to_csv('/Users/pinchichen/2025S lab/AI drug project/code/model performance metrics.csv', mode='a', header=False)

In [20]:
# append the performance to the csv file
df = {
    'Model':['XGB','XGB','XGB'],
    'Dataset':['Train','Validation','Test'],
    'MSE':[train_mse_mols,val_mse_mols,test_mse_mols],
    'RMSE':[train_rmse_mols,val_rmse_mols,test_rmse_mols],
    'MAE':[train_mae_mols,val_mae_mols,test_mae_mols],
    'R2':[train_r2_mols,val_r2_mols,test_r2_mols],
    'Pearson':[train_pearson_corr_mols,val_pearson_corr_mols,test_pearson_corr_mols],
    'Median_AE':[train_median_ae_mols,val_median_ae_mols,test_median_ae_mols],
    'Explained_VAR':[train_explained_var_mols,val_explained_var_mols,test_explained_var_mols],
    'Dataspliting Mode':['cold mols','cold mols','cold mols']
}
df = pd.DataFrame(df)

df.to_csv('/Users/pinchichen/2025S lab/AI drug project/code/model performance metrics.csv', mode='a', header=False)

In [21]:
# save the model
from joblib import dump, load


dump(xgb, '/Users/pinchichen/2025S lab/AI drug project/code/trained_model/cold_protein/XGBoost model_cold_protein.joblib')
dump(xgb_2,'/Users/pinchichen/2025S lab/AI drug project/code/trained_model/cold_mols/XGBoost model_cold_mols.joblib')

# load the model
#xgb_loaded = load('XGBoost model.joblib')

['/Users/pinchichen/2025S lab/AI drug project/code/trained_model/cold_mols/XGBoost model_cold_mols.joblib']